# Predictions
Prediction of sales is the central task in this challenge. you want to predict daily sales in various stores up to 6 weeks ahead of time. This will help the company plan ahead of time. 

The following steps outline the various sub tasks needed to effectively do this: 


In [1]:
# importing of libraries
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
import os,sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from timeseries import TimeSeries
from clean import Clean
sns.set()
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams["figure.figsize"] = (12, 8)
pd.set_option('display.max_columns', None)

/home/martin/Documents/pharm_sales/notebooks


# Preprocessing
It is important to process the data into a format where it can be fed to a machine learning model. This typically means converting all non-numeric columns to numeric, handling NaN values and generating new features from already existing features. 

In our case, you have a few datetime columns to preprocess. you can extract the following from them:
weekdays
weekends 
number of days to holidays
Number of days after holiday
Beginning of month, mid month and ending of month
(think of more features to extract), extra marks for it
            
As a final thing, you have to scale the data. This helps with predictions especially when using machine learning algorithms that use Euclidean distances. you can use the standard scaler in sklearn for this.


In [2]:
store = pd.read_csv("../data/store.csv")
df = pd.read_csv("../data/train.csv")
clean_df = Clean(df)
clean_df.merge_df(store,'Store')
clean_df.save(name='../data/unclean_train.csv')
clean_df.drop_missing_values()
clean_df.fix_outliers('Sales',25000)
clean_df.remove_unnamed_cols()
clean_df.transfrom_time_series("Store","Date")
clean_df.save(name="../data/training.csv")


/tmp/ipykernel_8514/2395586539.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/train.csv")
2022-05-24 16:21:14,902:logger:Successfully initialized clean class
2022-05-24 16:21:15,303:logger:Successfully merged the dataframe
2022-05-24 16:21:25,238:logger:Successfully saved the dataframe
2022-05-24 16:21:27,000:logger:Successfully dropped the columns with missing values
2022-05-24 16:21:27,002:logger:Successfully stored the features
2022-05-24 16:21:27,087:logger:Successfully handled outliers
2022-05-24 16:21:27,126:logger:Successfully removed columns with head unnamed
2022-05-24 16:21:27,816:logger:Successfully transformed data to time series data
2022-05-24 16:21:35,492:logger:Successfully saved the dataframe


In [3]:
df = pd.read_csv("../data/test.csv")
clean_df = Clean(df)
clean_df.merge_df(store,'Store')
clean_df.save(name='../data/unclean_test.csv')
clean_df.drop_missing_values()
clean_df.fix_outliers('Sales',25000)
clean_df.remove_unnamed_cols()
clean_df.transfrom_time_series("Store","Date")
clean_df.get_df().drop('Id',axis=1,inplace=True)
clean_df.save(name="../data/testing.csv")

2022-05-24 16:21:35,728:logger:Successfully initialized clean class
2022-05-24 16:21:35,746:logger:Successfully merged the dataframe
2022-05-24 16:21:36,148:logger:Successfully saved the dataframe
2022-05-24 16:21:36,202:logger:Successfully dropped the columns with missing values
2022-05-24 16:21:36,207:logger:Successfully stored the features
2022-05-24 16:21:36,208:logger:Successfully handled outliers
2022-05-24 16:21:36,214:logger:Successfully removed columns with head unnamed
2022-05-24 16:21:36,298:logger:Successfully transformed data to time series data
2022-05-24 16:21:36,617:logger:Successfully saved the dataframe


Label Encoding

In [4]:
train = pd.read_csv("../data/training.csv")
test = pd.read_csv("../data/testing.csv")
clean_df.label_encoding(train,test)

2022-05-24 16:21:55,854:logger:Successfully stored the features
2022-05-24 16:21:56,353:logger:Successfully encoded your categorical data


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Day,Month,Year,DayOfYear,WeekOfYear
7352,21,2,0,0,0,0,0,1,2,1,550.000,10.000,1999.000,1,45.000,2009.000,0,1,1,2013,1,1
7353,21,3,3680,393,1,0,1,1,2,1,550.000,10.000,1999.000,1,45.000,2009.000,0,2,1,2013,2,1
7354,21,4,3975,422,1,0,1,1,2,1,550.000,10.000,1999.000,1,45.000,2009.000,0,3,1,2013,3,1
7355,21,5,4471,464,1,0,1,1,2,1,550.000,10.000,1999.000,1,45.000,2009.000,0,4,1,2013,4,1
7356,21,6,5034,494,1,0,1,0,2,1,550.000,10.000,1999.000,1,45.000,2009.000,0,5,1,2013,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323328,1109,1,8082,622,1,1,1,0,2,0,3490.000,4.000,2011.000,1,22.000,2012.000,0,27,7,2015,208,31
323329,1109,2,6478,543,1,1,1,0,2,0,3490.000,4.000,2011.000,1,22.000,2012.000,0,28,7,2015,209,31
323330,1109,3,6212,500,1,1,1,0,2,0,3490.000,4.000,2011.000,1,22.000,2012.000,0,29,7,2015,210,31
323331,1109,4,5520,466,1,1,1,0,2,0,3490.000,4.000,2011.000,1,22.000,2012.000,0,30,7,2015,211,31
